In [5]:
!npm install localtunnel


up to date, audited 23 packages in 23s

3 packages are looking for funding
  run `npm fund` for details

2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.


In [ ]:
%%writefile app.py
import streamlit as st
import plotly.express as px
import pandas as pd
from PIL import Image
import random

# Nuevo estilo visual
st.markdown("""
    <style>
        :root {
            --primary: #00bfa6;
            --secondary: #ffffff;
            --accent: #ff6b6b;
            --dark-bg: #1e1e2f;
            --glass: rgba(255, 255, 255, 0.1);
            --glass-border: rgba(255, 255, 255, 0.2);
        }

        body, .stApp {
            background-color: var(--dark-bg);
            font-family: 'Segoe UI', 'Roboto', sans-serif;
        }

        .titulo-principal {
            font-size: 2.8rem;
            color: var(--primary);
            text-align: center;
            margin: 1.5rem 0;
            font-weight: bold;
        }

        .info-box {
            background: var(--glass);
            backdrop-filter: blur(10px);
            border: 1px solid var(--glass-border);
            padding: 1.5rem;
            border-radius: 15px;
            color: var(--secondary);
        }

        .data-card {
            background: var(--glass);
            border: 1px solid var(--glass-border);
            padding: 1rem;
            border-radius: 12px;
            text-align: center;
            color: var(--secondary);
            transition: all 0.3s ease-in-out;
        }

        .data-card:hover {
            transform: scale(1.03);
            box-shadow: 0 8px 16px rgba(0,0,0,0.3);
        }

        .stButton>button {
            background-color: var(--primary) !important;
            color: var(--dark-bg) !important;
            border: none;
            border-radius: 10px;
            font-weight: bold;
            padding: 0.5rem 1rem;
        }

        .stButton>button:hover {
            background-color: var(--accent) !important;
            color: white !important;
        }

        .sidebar .sidebar-content {
            background: #10101a;
            color: white;
        }

        .subtitulo {
            color: var(--secondary);
            font-size: 1.4rem;
            border-left: 4px solid var(--primary);
            padding-left: 0.5rem;
            margin-top: 1rem;
        }

        img.Denver-image {
            border-radius: 15px;
            box-shadow: 0 4px 8px rgba(0,0,0,0.5);
            transition: transform 0.3s ease;
        }

        img.Denver-image:hover {
            transform: scale(1.03);
        }
    </style>
""", unsafe_allow_html=True)

@st.cache_resource
def cargar_datos():
    datos = pd.read_csv("DenveLim.csv")
    columnas_numericas = datos.select_dtypes(['float', 'int']).columns
    columnas_texto = datos.select_dtypes(['object']).columns
    tipos_habitacion = datos['room_type'].unique()
    return datos, columnas_numericas, columnas_texto, tipos_habitacion, datos[columnas_numericas]

def cargar_imagenes():
    return [
        "Denver.jpeg",
        "https://images.unsplash.com/photo-1606830733746-2d0e5853a98b",
        "https://images.unsplash.com/photo-1606830733746-2d0e5853a98b"
    ]

def mostrar_imagen():
    imagen = random.choice(imagenes_Denver)
    st.markdown(f"""
        <div style='text-align: center;'>
            <img src='{imagen}' class='Denver-image' style='max-height: 300px; width: 100%; object-fit: cover;'>
        </div>
    """, unsafe_allow_html=True)

# Datos e imágenes
imagenes_Denver = cargar_imagenes()
df, columnas_numericas, columnas_texto, tipos_habitacion, df_numerico = cargar_datos()

# Menú lateral
vista = st.sidebar.selectbox("Selecciona vista", [
    "Inicio", "Exploración de Datos", "Análisis por Habitación",
    "Distribuciones", "Relaciones", "Comparativas"])

if vista == "Inicio":
    mostrar_imagen()
    st.markdown('<div class="titulo-principal">Análisis de Alojamientos en Denver</div>', unsafe_allow_html=True)
    st.markdown("""
        <div class='info-box'>
            <h3>Bienvenido al Dashboard de Denver</h3>
            <p>Explora y analiza datos de alojamientos.</p>
            <ul>
                <li>📈 Tendencias</li>
                <li>🏠 Tipos de habitaciones</li>
                <li>💰 Precios</li>
                <li>🔍 Variables</li>
            </ul>
        </div>
    """, unsafe_allow_html=True)
    col1, col2, col3 = st.columns(3)
    with col1:
        st.markdown(f"<div class='data-card'><h4>Total Alojamientos</h4><h2>{len(df):,}</h2></div>", unsafe_allow_html=True)
    with col2:
        st.markdown(f"<div class='data-card'><h4>Tipos Habitación</h4><h2>{len(tipos_habitacion)}</h2></div>", unsafe_allow_html=True)
    with col3:
        st.markdown(f"<div class='data-card'><h4>Precio Medio</h4><h2>€{df['price'].mean():.2f}</h2></div>", unsafe_allow_html=True)
    st.markdown('<div class="subtitulo">Muestra de Datos</div>', unsafe_allow_html=True)
    st.dataframe(df.head(10))

elif vista == "Exploración de Datos":
    mostrar_imagen()
    st.markdown('<div class="titulo-principal">Exploración de Datos</div>', unsafe_allow_html=True)
    if st.sidebar.checkbox("Ver datos", True):
        st.dataframe(df)
    if st.sidebar.checkbox("Columnas"):
        col1, col2 = st.columns(2)
        with col1: st.write("Numéricas", columnas_numericas)
        with col2: st.write("Texto", columnas_texto)
    if st.sidebar.checkbox("Estadísticas"):
        st.dataframe(df.describe())

elif vista == "Análisis por Habitación":
    mostrar_imagen()
    st.markdown('<div class="titulo-principal">Análisis por Habitación</div>', unsafe_allow_html=True)
    variables = st.sidebar.multiselect("Variables", columnas_numericas, default=columnas_numericas[:2])
    tipo = st.sidebar.selectbox("Tipo de Habitación", tipos_habitacion)
    datos = df[df['room_type'] == tipo]
    if variables:
        fig = px.line(datos, x=datos.index, y=variables, title=f"Tendencia - {tipo}")
        st.plotly_chart(fig)

elif vista == "Distribuciones":
    mostrar_imagen()
    st.markdown('<div class="titulo-principal">Distribuciones</div>', unsafe_allow_html=True)
    cat = st.sidebar.selectbox("Categoría", columnas_texto)
    num = st.sidebar.selectbox("Valor", columnas_numericas)
    fig = px.pie(df, names=cat, values=num, hole=0.3)
    st.plotly_chart(fig)

elif vista == "Relaciones":
    mostrar_imagen()
    st.markdown('<div class="titulo-principal">Relaciones</div>', unsafe_allow_html=True)
    x = st.sidebar.selectbox("Eje X", columnas_numericas)
    y = st.sidebar.selectbox("Eje Y", columnas_numericas)
    color = st.sidebar.selectbox("Color", ['None'] + list(columnas_texto))
    fig = px.scatter(df, x=x, y=y, color=None if color == 'None' else color, trendline='ols')
    st.plotly_chart(fig)

elif vista == "Comparativas":
    mostrar_imagen()
    st.markdown('<div class="titulo-principal">Comparativas</div>', unsafe_allow_html=True)
    cat = st.sidebar.selectbox("Categoría", columnas_texto)
    num = st.sidebar.selectbox("Valor", columnas_numericas)
    func = st.sidebar.selectbox("Agregación", ['sum', 'mean', 'median', 'count'])
    agrupado = df.groupby(cat)[num].agg(func).reset_index().sort_values(num, ascending=False)
    fig = px.bar(agrupado, x=cat, y=num, text=num)
    st.plotly_chart(fig)

st.markdown("""
    <div style="text-align: center; margin-top: 2rem; color: gray;">
        © 2025 Dashboard Denver - Hecho con Streamlit
    </div>
""", unsafe_allow_html=True)


Overwriting app.py
